In [1]:
import rospy
import rosbag
import numpy as np
import cv2
from cv_bridge import CvBridge
import tensorflow as tf

from sensor_msgs.msg import Image
from geometry_msgs.msg import Twist
#from bebop_msgs.msg import Ardrone3PilotingStateAltitudeChanged

IMG_SIZE = 100

In [2]:
# Data loading
bag = rosbag.Bag('test_2020-06-29-16-38-29.bag')
bridge = CvBridge()

t1 = rospy.Time(1593466714, 560337720)
t2 = rospy.Time(1593466728, 846956538)

img = []
flow = []

for topic, msg, t in bag.read_messages(topics=['/bebop/image_raw', '/bebop/cmd_vel']):
    if t >= t1 and t < t2:
        if topic == '/bebop/image_raw':
            img.append(bridge.imgmsg_to_cv2(msg, 'bgr8'))
            if len(img) > 1:
                prev = cv2.cvtColor(img[-2],cv2.COLOR_BGR2GRAY)
                curr = cv2.cvtColor(img[-1],cv2.COLOR_BGR2GRAY)
                prev = cv2.resize(prev, (IMG_SIZE, IMG_SIZE))
                curr = cv2.resize(curr, (IMG_SIZE, IMG_SIZE))
                flow_s = cv2.calcOpticalFlowFarneback(prev, curr, None, pyr_scale=0.5, levels=3, 
                                                           winsize=15, iterations=3, poly_n=5, poly_sigma=1.2, 
                                                           flags=0)
                flow.append(flow_s)

print(len(img))
print(len(flow))

428
427


In [3]:
# Data scaling
out_min = -1
out_max = 1

X = np.array(flow)
X_scaled = np.copy(X)

x_min = np.min(X)
x_max = np.max(X)

cnt = 0
for xi in X:
    x_std = (xi - x_min)/(x_max - x_min)
    x_scld = x_std*(out_max - out_min) + out_min
    X_scaled[cnt] = x_scld
    cnt = cnt + 1

print(X_scaled.shape)
print(np.min(X_scaled))
print(np.max(X_scaled))

(427, 100, 100, 2)
-1.0
1.0


In [4]:
# Prediction
CATEGORIES = ["Normal", "GE"]

model = tf.keras.models.load_model("32x2conv-0dense.model")

prediction = model.predict(X_scaled)

print(prediction)
print(CATEGORIES[int(prediction[0])])

[[1.45990625e-05]
 [1.24909939e-05]
 [1.37842671e-05]
 [1.36977260e-05]
 [1.24144208e-05]
 [1.34877073e-05]
 [1.47484743e-05]
 [1.34535221e-05]
 [1.46641405e-05]
 [1.21348030e-05]
 [1.39174790e-05]
 [1.21752582e-05]
 [1.31639154e-05]
 [1.32772093e-05]
 [1.42989838e-05]
 [1.30463859e-05]
 [1.29159789e-05]
 [1.26919431e-05]
 [1.35826185e-05]
 [1.39169224e-05]
 [1.33333051e-05]
 [1.37576199e-05]
 [1.40754355e-05]
 [1.28563534e-05]
 [1.39544127e-05]
 [1.35080199e-05]
 [1.24231246e-05]
 [1.22378633e-05]
 [1.01781661e-05]
 [1.49331545e-05]
 [1.04425917e-05]
 [1.62811248e-05]
 [1.42333256e-05]
 [9.81895391e-06]
 [2.17676970e-05]
 [1.16009905e-05]
 [8.32205296e-06]
 [1.45103822e-05]
 [1.86843972e-05]
 [8.78635728e-06]
 [1.24246762e-05]
 [2.19881877e-05]
 [1.31742017e-05]
 [1.34327156e-05]
 [1.60981399e-05]
 [7.03524074e-06]
 [1.55400139e-05]
 [2.24334053e-05]
 [1.22267111e-05]
 [1.45965987e-05]
 [1.24820035e-05]
 [1.33568619e-05]
 [1.25126217e-05]
 [1.38903497e-05]
 [2.15079012e-06]
 [2.977791

In [5]:
# Create video
height, width = img[0].shape[0:2]
#out = cv2.VideoWriter('project.avi', cv2.VideoWriter_fourcc(*'DIVX'), 30, size)
out = cv2.VideoWriter('project.avi', cv2.VideoWriter_fourcc(*'MJPG'), 30, (width, height), True)

for i in range(len(img)-1):
    img_c = img[i+1].copy()
    
    lbl = CATEGORIES[int(prediction[i])]
    cv2.putText(img_c, lbl, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    out.write(img_c)
    
out.release()